In [0]:
# 구글 드라이브 마운트, 주피터에선 사용 X
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 필수 라이브러리 import
%tensorflow_version 1.x
import math
import os
import pickle
import numpy as np

TensorFlow 1.x selected.


In [0]:
# 변수 설정
normal_path = '/content/gdrive/My Drive/캡스톤/ben/' #정상파일 경로
mal_path = '/content/gdrive/My Drive/캡스톤/mnemonic/' #악성파일 경로
model_path = '/content/gdrive/My Drive/캡스톤/' #모델 가중치 저장 경로
mc_len = 0
block_id = {}
mnemonic_table = {}
mnemonic_id = {}

In [0]:
def pickle_load(path):
  f = open(path, 'rb')
  data = pickle.load(f)
  f.close()
  return data

def pickle_dump(path, data):
  f = open(path, 'wb')
  pickle.dump(data, f)
  f.close()

In [0]:
def mnemonic_count(blocks):
  global mc_len
  for block in blocks:
    tf = {}
    for mnemonics in block:
      len_mnemonics = len(mnemonics)
      for mnemonic in mnemonics:
        if mnemonic not in tf:
          tf[mnemonic] = 1 / len_mnemonics
        else:
          tf[mnemonic] += 1 / len_mnemonics
    block_id[mc_len] = tf
    mc_len += 1

def data_load_for_word_count(path, start, end):
  file_names = os.listdir(path)
  file_names = file_names[start:end]
  for index, file_name in enumerate(file_names):
    print("\r{} / {}".format(index + 1, len(file_names)), end="")
    try:
      mnemonic = pickle_load(path + file_name)
      mnemonic_count(mnemonic)
    except:
      pass
  print()

In [0]:
def fw_indexing():
  for index in block_id:
    if index % 1000 == 0:
      print("\rfw_indexing part 1: {} / {}".format(index + 1, len(block_id)), end="")
    for mnemonic in block_id[index]:
      if mnemonic in mnemonic_table:
        mnemonic_table[mnemonic] += 1
      else:
        mnemonic_table[mnemonic] = 1
  print()

  smnemonic_table = sorted(mnemonic_table, key=lambda k : mnemonic_table[k])
  for mnemonic in smnemonic_table:
    if mnemonic_table[mnemonic] > 10000:
      break
    else:
      del mnemonic_table[mnemonic]
  
  len_block_id = len(block_id)
  for index in block_id:
    if index % 1000 == 0:
      print("\rfw_indexing part 2: {} / {}".format(index + 1, len(block_id)), end="")
    for mnemonic in block_id[index]:
      if mnemonic in mnemonic_table:
        block_id[index][mnemonic] *= math.log10(len_block_id / (1 + mnemonic_table[mnemonic]))
      else:
        block_id[index][mnemonic] *= math.log10(len_block_id)
  print()

In [0]:
def bk_indexing():
  for index in block_id:
    if index % 1000 == 0:
      print("\rbk_indexing: {} / {}".format(index + 1, len(block_id)), end="")
    for mnemonic in block_id[index]:
      if mnemonic in mnemonic_id:
        mnemonic_id[mnemonic][index] = block_id[index][mnemonic]
      else:
        mnemonic_id[mnemonic] = {}
        mnemonic_id[mnemonic][index] = block_id[index][mnemonic]
  print()

In [0]:
data_load_for_word_count(normal_path, 0, 1000)
pickle_dump(model_path + 'block_id_1000.dat', block_id)
fw_indexing()
pickle_dump(model_path + 'mnemonic_table_1000.dat', mnemonic_table)
bk_indexing()
pickle_dump(model_path + 'mnemonic_id_1000.dat', mnemonic_id)

1000 / 1000
fw_indexing part 1: 5773001 / 5773986
fw_indexing part 2: 5773001 / 5773986
bk_indexing: 5773001 / 5773986


In [0]:
block_id = pickle_load(model_path + 'block_id_1000.dat')
mnemonic_table = pickle_load(model_path + 'mnemonic_table_1000.dat')
mnemonic_id = pickle_load(model_path + 'mnemonic_id_1000.dat')

In [0]:
print(len(block_id))
print(len(mnemonic_table))
print(len(mnemonic_id))

5773986
170
1363


In [0]:
def search(blocks):
  ret = []
  search_block_id = {}
  len_block_id = len(block_id)
  for block in blocks:
    tf = {}
    tf_size = 0
    for mnemonics in block:
      len_mnemonics = len(mnemonics)
      for mnemonic in mnemonics:
        if mnemonic in tf:
          tf[mnemonic] += 1 / len_mnemonics
        else:
          tf[mnemonic] = 1 / len_mnemonics
    numerator = {}
    denominator = {}
    for mnemonic in tf:
      if mnemonic in mnemonic_table:
        tf_idf = tf[mnemonic] * math.log10(len_block_id / (1 + mnemonic_table[mnemonic]))
      else:
        tf_idf = tf[mnemonic] * math.log10(len_block_id)
      tf_size += tf_idf ** 2
      for index in mnemonic_id[mnemonic]:
        if index in numerator:
          numerator[index] += mnemonic_id[mnemonic][index] * tf_idf
        else:
          numerator[index] = mnemonic_id[mnemonic][index] * tf_idf
        if index in denominator:
          denominator[index] += mnemonic_id[mnemonic][index] ** 2
        else:
          denominator[index] = mnemonic_id[mnemonic][index] ** 2
    cos_sim = {}
    for index in numerator:
      cos_sim[index] = numerator[index] / math.sqrt(denominator[index] * tf_size)
    ret.append(min(cos_sim.values()))
  ret = np.array(ret)
  return ret.mean()

def evaluate(path, start, end):
  scores = []
  file_names = os.listdir(path)
  file_names = file_names[start:end]
  for index, file_name in enumerate(file_names):
    print("\r{} / {}".format(index + 1, len(file_names)), end="")
    try:
      blocks = pickle_load(path + file_name)
      score = search(blocks)
      print(score)
      scores.append(score)
    except Exception as e:
      print(e)
  print()
  return np.array(scores).mean()


In [0]:
normal_score = evaluate(normal_path, 1000, 1010)
mal_score = evaluate(mal_path, 0, 10)

1 / 100.11938581222020433
2 / 10'addiu'
3 / 100.34078392455139633
4 / 100.0041703784103595
5 / 100.30447029632744377
6 / 10

In [0]:
f = open(model_path + "evaluate.txt", 'w')
f.write(str(normal_score) + "\n")
f.write(str(mal_score) + "\n")
f.close()

NameError: ignored